In [1]:
!pip install tensorflow==2.3.2
!pip install tensorflow-gpu==2.3.2
!pip install keras==2.3.0

  Using cached numpy-1.18.5-cp37-cp37m-manylinux1_x86_64.whl (20.1 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.0
    Uninstalling numpy-1.19.0:
      Successfully uninstalled numpy-1.19.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tables 3.7.0 requires numpy>=1.19.0, but you have numpy 1.18.5 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


In [2]:
!pip3 install git+git://github.com/minimaxir/textgenrnn.git

  Cloning git://github.com/minimaxir/textgenrnn.git to /tmp/pip-req-build-kcvakfl9
  Running command git clone -q git://github.com/minimaxir/textgenrnn.git /tmp/pip-req-build-kcvakfl9


In [22]:
!pip install -U numpy==1.19

     |████████████████████████████████| 14.6 MB 4.2 MB/s 
  Attempting uninstall: numpy
    Found existing installation: numpy 1.18.5
    Uninstalling numpy-1.18.5:
      Successfully uninstalled numpy-1.18.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.3.2 requires numpy<1.19.0,>=1.16.0, but you have numpy 1.19.0 which is incompatible.
tensorflow-gpu 2.3.2 requires numpy<1.19.0,>=1.16.0, but you have numpy 1.19.0 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


In [3]:
import tensorflow
from textgenrnn import textgenrnn
import numpy as np

In [4]:
!pip install langdetect

In [5]:
import pandas as pd
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from langdetect import detect

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
data = pd.read_csv('/content/buzzfeed_headlines.csv')
data.sample(10)

,content,description,title
383,"Oh, to be stuck in traffic behind a ""Butt Snor...","Oh, to be stuck in traffic behind a ""Butt Snor...",21 Hilarious Bumper Stickers That Are Guarante...
520,Each of these has eight different color modes ...,These custom floral coasters are going to make...,36 Products To Add Subtle Personal Touches To ...
31,"Netflix\r\nI won't say too much else because, ...",Let's have a moment of silence for all those b...,"22 Funny And Relatable Reactions To ""The Woman..."
405,When you purchase it comes with one Rocketbook...,"You'll save money, too.View Entire Post ›",Feel Superior To Your Single-Use Plastic Frien...
651,Why cant I dress myself? Im an adult.,“Why can’t I dress myself? I’m an adult.”View ...,Tefi Pessoa Joked About Wearing Her Dress Inco...
723,"Cbs Photo Archive / CBS via Getty Images\r\n""I...","""It made me really sad.""View Entire Post ›",Adele Recalled Feeling Ashamed During Her Divo...
195,The coronavirus pandemic is still impacting tr...,Portugal has been getting a lot of well-deserv...,31 Photos Of Portugal That Are Almost As Good ...
802,"It'll remove rust, tarnish, stains, and minera...","""The Princess Diaries"" soundtrack made several...",30 Products That Prove Miracles Happen Once In...
840,"Dog A DayThe cutest pups, every day in your inbox",Not sure I would be this brave!View Entire Post ›,Here's How Australians React When They Spot A ...
274,We hope you love the products we recommend! Al...,You’re spending a lot of time at home so might...,28 Home Decor Products You Need That’ll Instan...


In [7]:
data.drop(['content', 'description'], inplace=True, axis=1)
data.sample(460)

,title
553,"15 Times Steve Harvey Absolutely Lost It At ""F..."
6,Directors Must Really Love These 17 Pairs Of A...
831,"Chances Are, Your Birth Control Comes With A L..."
206,17 Random Unsettling Facts That Will Make Your...
636,39 Hard-Working Products Anyone Who Identifies...
...,...
639,39 Products With Before And Afters That’ll Mak...
194,"I Cook At Home Nearly Every Day, And These Are..."
865,25 Products With Disgusting-Yet-Satisfying Bef...
370,I Ranked 32 Movie And TV Wedding Dresses By Ho...


In [8]:
def detect_lang(txt):
    try:
        return detect(txt)
    except:
        return np.nan
    
data['language'] = data.title.apply(detect_lang)
data = data[data.language == "en"]
data = data.drop('language', axis=1)

In [9]:
def simple_preprocess(df):
  df['title'] = df['title'].str.lower()
  df['title'] = df['title'].str.replace('[^a-zA-Z0-9]', ' ') # replace non-alphanumeric characters with spaces
  return df

In [10]:
def preprocess(df):
  stop_words = stopwords.words('english')
  df['title'] = df['title'].str.lower()
  df['title'] = df['title'].str.replace('[^a-zA-Z0-9]', ' ') # replace non-alphanumeric characters with spaces
  df['title'] = df['title'].str.replace('\s+', ' ') # replace multiple spaces with a single space
  df['title'] = df['title'].str.strip() # remove leading and trailing spaces
  df['title'] = df['title'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))

  return df

In [11]:
data = simple_preprocess(data)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  This is separate from the ipykernel package so we can avoid doing imports until


In [12]:
# data = preprocess(data)

In [13]:
data.shape

(941, 1)

In [14]:
data.sample(10)

,title
954,here s an update on the most memorable antm ...
712,jamie foxx said he doesn t plan on getting mar...
447,21 all american moments that prove spencer a...
843,here s where you ve seen the cast of locke ...
662,what questions does squid game season two ne...
655,these 35 bad parents have zero excuses for the...
817,have you been treated differently at work beca...
516,if you want to feel just a little bit more l...
423,42 baby products new parents might want to inv...
377,21 romantic comedies for people who hate romcoms


In [15]:
data_arr = data['title'].to_numpy()
model = textgenrnn()

In [16]:
EPOCHS = 10
PERCENT_TRAINING = 0.75

In [17]:
model_cfg = {
    'word_level': True,
    'rnn_size': 128,
    'rnn_layers': 3,
    'rnn_bidirectional': False,
    'max_length': 5,
    'max_words': 50
}

train_cfg = {
    'line_delimited': False,
    'num_epochs': EPOCHS,
    'gen_epochs': 2,
    'train_size': 1,
    'dropout': 0.2,
    'validation': False,
    'is_csv': False
}

In [18]:
model.train_on_texts(
    data_arr,
    num_epochs=EPOCHS,
    new_model=True,
    gen_epochs=train_cfg['gen_epochs'],
    batch_size=64,
    train_size=train_cfg['train_size'],
    dropout=train_cfg['dropout'],
    validation=train_cfg['validation'],
    is_csv=train_cfg['is_csv'],
    rnn_layers=model_cfg['rnn_layers'],
    rnn_bidirectional=model_cfg['rnn_bidirectional'],
    max_length=model_cfg['max_length'],
    dim_embedding=200,
    word_level=model_cfg['word_level']
)
model.save(weights_path=f"buzzfeed_headline_model{EPOCHS}_epochs_{PERCENT_TRAINING}.hdf5")

Training new model w/ 3-layer, 128-cell LSTMs
Training on 16,060 word sequences.
Epoch 1/10
250/250 [==============================] - 19s 75ms/step - loss: 6.8605
Epoch 2/10
250/250 [==============================] - ETA: 0s - loss: 5.9775####################
Temperature: 0.2
####################
people are sharing most most their lesson they

people are sharing their the their their their

23 people are sharing most they they they ve

####################
Temperature: 0.5
####################
if you feel your home

23 and i m m even a heartthrob and i m m m if you re to ll your tip

people are sharing things their their their australia

####################
Temperature: 1.0
####################
20 famous facts you absolutely sleep freaking 10 pirates on of interesting

if you re to fitness go hard s haunting out

14 people taxes stories times save landlord ll recalled her die man will christmas earth

250/250 [==============================] - 25s 99ms/step - loss: 5.9775
Epoch 3/10


In [19]:
# defining a range for temperature - this gauges how obscure the generated headlines are
import numpy as np

temps = np.linspace(0.1, 1, 10)
model.generate_samples(n=3, temperatures=temps)
model.generate_samples(n=5, temperatures=temps, prefix="crazy things")

####################
Temperature: 0.1
####################
if you ve lived through a natural disaster tell us what was a warning when you don t get enough of attention to your guess your space

here s what you should do instead

people are sharing the most paranormal thing that has ever happened to them and after reading these i can t sleep

####################
Temperature: 0.2
####################
here s what you should do instead

people are sharing things they completely splurged on that were 100 worth it

if you want results check out these 31 products with impressive before and after photos

####################
Temperature: 0.30000000000000004
####################
i m sorry but if you did literally any these 40 things growing up congratulations you re officially old

people are sharing their silliest and pettiest reasons for why

people are sharing their silliest and pettiest reasons for why they ve had to set up with anthony hopkins because she couldn t separate him from hannib

In [20]:
model_2 = textgenrnn()

In [21]:
data_list = list(data['title'])

In [22]:
print(data_list)

['16 wholesome details from tv and movies that show they were quite literally made with love', 'jacob elordi has been living the dream filming passionate love scenes with sydney sweeney on euphoria and he knows it', 'people are sharing things that helped them get a better work life balance and it s so refreshing', '7 actors who were considered too ugly for a role and 8 who weren t ugly enough', 'brian cox says he turned down a role in pirates of the caribbean ', 'eat at this dessert buffet like a toddler at a birthday party and i ll tell you which bsc character you remind me of', 'directors must really love these 17 pairs of actors together since they keep getting cast as costars', 'all of the major pop culture happenings this week from celebrity tributes to new trailers', 'aussies are sharing how their experiences overseas have changed their view on australia and the responses are fascinating', 'telling you not to discuss your pay with coworkers and 18 other red flags in the workplace

In [23]:
model_2.train_on_texts(data_list, num_epochs=10)
model_2.generate(5)

Training on 82,540 character sequences.
Epoch 1/10
644/644 [==============================] - ETA: 0s - loss: 1.8573####################
Temperature: 0.2
####################
all these products are the so compled the so the song to the and it sharing the so the song for the sexured the movie and it s and it s and the show them

all the so show the so the the sured and and it s and the song and the song and the sexured the couple the so show the song and and i made them

all and all the song that have all the sexured the song and all the show and and i made the song 

####################
Temperature: 0.5
####################
21 people that ll get the comming the comming and we lean their some in the money the sexured out the the movie and you weren while of the solve movies in the scroet us with and all up them

up and a look show all then a love the sine in these products and is of these sard horry about so products who loved and the sexing the coople spart their products 

33 people 

 20%|██        | 1/5 [00:03<00:14,  3.68s/it]

32 subtle products that are sharing the unsam broken they were released to tnow your no holland 



 40%|████      | 2/5 [00:07<00:11,  3.86s/it]

 marvel destagram products that reveal your one belover realize and after prince than the same are season



...

21 skincare episodes that prove that are sharing the stranger and thhe role and more everything do that might ever you do any of their 2 and it s a all of the cream in character in the same and it s a street made and it s sunce as her turners and i m student and we ll reveal your scene i love the c



 80%|████████  | 4/5 [00:21<00:05,  5.48s/it]

david base new show you can not know which thing is the character incarextic



100%|██████████| 5/5 [00:29<00:00,  6.00s/it]

17 opial engagement worth the characters based on the camerate and it s a stranger and the reasons why their news and the characters who were replaced to the character you re actually had their stories about the most movie teen 

